# Q2 - Durable Purchases

Imports and set magics:

In [ ]:
# Import of packages
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as sm

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# Question 1

## Basic functions:

In [ ]:
def v2(m2,d,alpha,rho,chi):
    x = m2 - c
    return ((c**alpha*(d+chi*x)**(1-alpha))**(1-rho))/(1-rho)

def v1(a,d,Delta,v2_interp,r,m2,v2):
    m1 = a + d
     # a. v2 value, if low income
    m2_low = (1+r)*a + 1-Delta
    v2_low = v2_interp([m2_low])[0]

    # b. v2 value, if medium income
    m2_medium = (1+r)*a + 1
    v2_medium = v2_interp([m2_medium])[0]
    
    # c. v2 value, if high income
    m2_high = (1+r)*a + 1+Delta
    v2_high = v2_interp([m2_high])[0]
    
    # d. expected v2 value
    v2 = 1/3*v2_low + 1/3*v2_medium + 1/3*v2_high

    # e. total value
    return beta*v2


## Parameters:

In [ ]:
    rho = 2
    alpha = 0.8
    beta = 0.96
    r = 0.04
    Delta = 0.25
    chi = 0.9


## Solution functions:

In [71]:
def solve_period_2(alpha,rho,chi):

    # a. grids
    m2_vec = np.linspace(1e-8,10,100)
    v2_vec = np.empty(100,100)
    c_vec = np.empty(100,100)
    x_vec = np.empty(100,100)

    # b. solve for each m2 in grid
    for j,d in enumerate(d_vec):
        for i,m2 in enumerate(m2_vec):

        # i. objective
        obj = lambda c: -v2(c,rho,d,alpha,chi,m2)

        # ii. initial value (consume half)
        x0 = m2/2

        # iii. optimizer
        result = optimize.minimize_scalar(obj,x0,method='bounded',bounds=[1e-8,m2])

        # iv. save
        v2_vec[i,j] = -result.fun
        c_vec[i,j] = result.x
        x_vec[i,j] = m2 - result.x
        
    return m2_vec,v2_vec,c_vec,x_vec


IndentationError: expected an indented block (<ipython-input-71-9340e6a2c69c>, line 14)

In [ ]:
    # solve
m2_vec,c_vec,v2_vec,x_vec = solve_period_2(m2,alpha,rho,c,chi,d)

# illustration
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(1,2,1)
ax.plot(m2_vec,c_vec)
ax.set_xlabel('$m_2$')
ax.set_ylabel('$c$')
ax.set_title('non-durable consumption function in period 2')

ax = fig.add_subplot(1,2,2)
ax.plot(m2_vec,x_vec)
ax.set_xlabel('$m_2$')
ax.set_ylabel('$x$')
ax.set_title('durable consumption function in period 2')
ax.set_ylim([-40,1]);

ax = fig.add_subplot(1,2,2)
ax.plot(m2_vec,v2_vec)
ax.set_xlabel('$m_2$')
ax.set_ylabel('$v_2$')
ax.set_title('value function in period 2')
ax.set_ylim([-40,1]);

In [ ]:
v2_interp = interpolate.RegularGridInterpolator([m2_vec], v2_vec,bounds_error=False,fill_value=None)

In [ ]:
def solve_period_1(rho,beta,r,a,Delta,v2_interp):

    # a. grids
    m1_vec = np.linspace(1e-8,4,100)
    v1_vec = np.empty(100)
    c1_vec = np.empty(100)
    
    # b. solve for each m1 in grid
    for i,m1 in enumerate(m1_vec):
        
        # i. objective
        obj = lambda c: -v1(m1,d,a)
        
        # ii. initial guess (consume half)
        x0 = m1*1/2
        
        # iii. optimize
        result = optimize.minimize_scalar(obj,x0,method='bounded',bounds=[1e-8,m1])
        
        # iv. save
        v1_vec[i] = -result.fun
        c_vec[i] = result.x
     
    return m1_vec,v1_vec,c_vec

# solve
m1_vec,v1_vec,c_vec = solve_period_1(rho,beta,r,Delta,v2_interp)

# illustrate
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(1,2,1)
ax.plot(m1_vec,c1_vec)
ax.set_xlabel('$m_1$')
ax.set_ylabel('$c_1$')
ax.set_title('consumption function in period 1')

ax = fig.add_subplot(1,2,2)
ax.plot(m1_vec,v1_vec)
ax.set_xlabel('$m_1$')
ax.set_ylabel('$c_1$')
ax.set_title('value function in period 1')
ax.set_ylim([-40,1]);

## Question A:

# Conclusion